# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback
* Original Data 3148

In [22]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-25","2019-03-01",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-25 to 2019-03-01
997


In [23]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [24]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [25]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Cannabis may be coming to a mall near you: rol...   
1  Target is taking a page out of Amazon's playbo...   
2  Stocks trade mixed on conflicting reports on U...   
3  Stocks making the biggest moves midday: Gap, T...   
4     The difficulties of making online delivery pay   

                                         description  
0  Cannabis is going upscale. The plant,  which i...  
1  Target has started to add third-party sellers ...  
2  NEW YORK (AP) — The stock market shook off an ...  
3  These are the stocks posting the largest moves...  
4  Grocers are eager to fulfil customer orders bu...  


In [26]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 578/578 [06:20<00:00,  3.08s/it]


In [27]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:03<00:00, 153.25it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 578/578 [00:03<00:00, 145.01it/s]


     cannabis     going  mainstream   upscale     plant     whose   species  \
0       1.250  0.037594    0.230769  0.166667  0.086957  0.046154  0.333333   
1       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
2       0.000  0.007519    0.000000  0.000000  0.000000  0.000000  0.000000   
3       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
4       0.000  0.015038    0.000000  0.000000  0.000000  0.000000  0.000000   
5       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
6       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
7       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
8       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
9       0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
10      0.000  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000   
11      0.000  0.000000    0.000000  0.000000  0.000

[578 rows x 22197 columns]


In [28]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [29]:
import pandas as pd
articles = pd.DataFrame(frontpage['articles'])
topterms= pd.DataFrame(frontpage['topterms'])
articles.columns

Index(['date', 'origContent', 'related_articles', 'source', 'tags', 'title'], dtype='object')

In [30]:
titles = pd.DataFrame(articles['tags'])

In [31]:
pd.DataFrame.to_excel(articleDB, 'Main Version-original data - 25-01.xlsx')